## 1. Imports

> CLEAN THIS PLEASE

In [129]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#Scaling
from sklearn.preprocessing import MinMaxScaler

 #Correlation Heatmap
from matplotlib.colors import LinearSegmentedColormap

#Statistical Test
from scipy import stats
from sklearn.impute import SimpleImputer

#Models
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split

from sklearn.model_selection import KFold, RepeatedKFold, cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.linear_model import Ridge

pd.set_option('display.max_columns', None)


import pandas as pd  # For creating and handling DataFrames
import numpy as np  # For numerical operations
from sklearn.feature_selection import chi2  # For Chi-square test
from scipy.stats import chi2_contingency  # For Cramér's V calculation

In [130]:
# Read the dataset
X_train_encoded = pd.read_csv('../project_data/X_train_encoded.csv', delimiter=',', index_col=0)
X_val_encoded = pd.read_csv('../project_data/X_val_encoded.csv', delimiter=',', index_col=0)

y_train = pd.read_csv('../project_data/y_train.csv',delimiter=',', index_col=0)
y_val= pd.read_csv('../project_data/y_val.csv', delimiter=',', index_col=0)

X_test_encoded = pd.read_csv('../project_data/X_test_encoded.csv',index_col=0)

In [131]:
# Binary classification dataset
X_train_encoded_bin = pd.read_csv('../project_data/X_train_encoded_binary.csv', delimiter=',', index_col=0)
X_val_encoded_bin = pd.read_csv('../project_data/X_val_encoded_binary.csv', delimiter=',', index_col=0)

y_train_bin = pd.read_csv('../project_data/y_train_binary.csv',delimiter=',', index_col=0)
y_val_bin= pd.read_csv('../project_data/y_val_binary.csv', delimiter=',', index_col=0)

X_test_encoded_bin = pd.read_csv('../project_data/X_test_encoded_binary.csv',index_col=0)

## 2. Preparing the data

### 2.1 Encoding multiclass target

> Encode the multiclass target and exclude 2. NON-COMP from the dataset, so it's easier to train

In [134]:
claim_injury_type_mapping = {
    '4. TEMPORARY': 0,
    '2. NON-COMP':7,
    '5. PPD SCH LOSS': 1,
    '3. MED ONLY': 2,
    '6. PPD NSL': 3,
    '1. CANCELLED': 4,
    '8. DEATH':5,
    '7. PTD': 6
}

y_train_encoded = y_train['Claim Injury Type'].map(claim_injury_type_mapping)
y_val_encoded = y_val['Claim Injury Type'].map(claim_injury_type_mapping)
y_val_encoded_final = y_val['Claim Injury Type'].map(claim_injury_type_mapping)
X_val_encoded_final = X_val_encoded.copy()

In [135]:

# Ensure y_train is a Series (if it's a DataFrame with one column, convert it to Series)
y_train = y_train.squeeze()  # This will convert DataFrame with a single column to a Series

# Create a mask for rows where '2. NON-COMP' is not present
mask = y_train != '2. NON-COMP'

# Apply the mask to filter out '2. NON-COMP'
y_train_not_encoded = y_train[mask]

# Print the shape of the filtered target
print("Shape of y_train_not_encoded:", y_train_not_encoded.shape)

# Check the unique classes in the filtered target to confirm that '2. NON-COMP' is removed
print("Classes in y_train_not_encoded:", y_train_not_encoded.unique())


Shape of y_train_not_encoded: (198056,)
Classes in y_train_not_encoded: ['5. PPD SCH LOSS' '3. MED ONLY' '4. TEMPORARY' '1. CANCELLED' '8. DEATH'
 '6. PPD NSL' '7. PTD']


In [136]:
# Create a mask for rows where y_train_encoded is not equal to 7 (non-comp class)
mask_train = y_train_encoded != 7

# Filter X_train_encoded and y_train_encoded based on the mask
X_train_encoded = X_train_encoded[mask_train]
y_train_encoded = y_train_encoded[mask_train]

# Apply the same mask to y_val_encoded and X_val_encoded
mask_val = y_val_encoded != 7
X_val_encoded = X_val_encoded[mask_val]
y_val_encoded = y_val_encoded[mask_val]

# Verify the shapes of the new filtered datasets
print("Shape of X_train_encoded_filtered:", X_train_encoded.shape)
print("Shape of y_train_encoded_filtered:", y_train_encoded.shape)
print("Shape of X_val_encoded_filtered:", X_val_encoded.shape)
print("Shape of y_val_encoded_filtered:", y_val_encoded.shape)

# Check unique classes in the target to confirm "non comp" is removed
print("Classes in y_train_encoded_filtered:", y_train_encoded.unique())
print("Classes in y_val_encoded_filtered:", y_val_encoded.unique())  # Corrected this line


Shape of X_train_encoded_filtered: (198056, 58)
Shape of y_train_encoded_filtered: (198056,)
Shape of X_val_encoded_filtered: (84881, 58)
Shape of y_val_encoded_filtered: (84881,)
Classes in y_train_encoded_filtered: [1 2 0 4 5 3 6]
Classes in y_val_encoded_filtered: [2 4 0 1 3 6 5]


### 2.2 Separate Numerical and Categorical

#### 2.2.1  Binary separation

> Separare the Numerical and Categorical Columns

In [137]:
num_columns_bin = ['Age at Injury', 
                   'IME-4 Count', 
                   'Number of Dependents',

                   'Industry Code_encoded_0',
                   'Industry Code_encoded_1', 

                   'WCIO Cause of Injury Code_encoded_0',
                   'WCIO Cause of Injury Code_encoded_1',

                   'WCIO Nature of Injury Code_encoded_0',
                   'WCIO Nature of Injury Code_encoded_1',

                   'WCIO Part Of Body Code_encoded_0', 
                   'WCIO Part Of Body Code_encoded_1',

                   'Industry Code_freq', 
                   'WCIO Cause of Injury Code_freq',
                   'WCIO Nature of Injury Code_freq',
                   'WCIO Part Of Body Code_freq',
                   'Carrier Type_freq', 
                   'Carrier Name_freq',
                   'Accident Datemonth',
                   'Accident Date_Season_Spring', 
                   'Accident Date_Season_Summer',
                   'Accident Date_Season_Winter',
                   'Days_between_C-2 Date_Accident Date_log',

                   'Days_between_Assembly Date_Accident Date_log',
                   'Average Weekly Wage Imputed_log']

cat_columns_bin = ['Carrier Type_Self-insured Private Entity',
                   'Carrier Type_Self-insured Public Entity', 
                   'Carrier Type_Special Funds',
                   'Carrier Type_State Insurance Fund', 
                   'C-3 Date_nabinary', 
                   'Average Weekly Wage_nabinary',
                   'First Hearing Date_nabinary',
                   'Alternative Dispute Resolution_binary',
                   'COVID-19 Indicator_binary',
                   'Attorney/Representative_binary']

# Create subsets
X_train_num_bin = X_train_encoded_bin[num_columns_bin]
X_train_cat_bin = X_train_encoded_bin[cat_columns_bin]

X_val_num_bin = X_val_encoded_bin[num_columns_bin]
X_val_cat_bin = X_val_encoded_bin[cat_columns_bin]

X_test_num_bin=X_test_encoded_bin[num_columns_bin]
X_test_cat_bin=X_test_encoded_bin[cat_columns_bin]

#### 2.2.2 Multiclass separation

> Drop the ones associated the the target class 2. NON-COMP

In [138]:
# Drop features associated with "non comp" in their name
columns_to_drop = [col for col in X_train_encoded.columns if "non-comp" in col.lower()]

# Drop the columns from the dataset
X_train_encoded = X_train_encoded.drop(columns=columns_to_drop)
X_val_encoded = X_val_encoded.drop(columns=columns_to_drop)  # If you have validation data
X_test_encoded = X_test_encoded.drop(columns=columns_to_drop)  # If you have test data

# Verify the remaining columns
print("Remaining columns after dropping 'non comp' features:")
print(len(X_train_encoded.columns))

Remaining columns after dropping 'non comp' features:
54


> Separate the numerical and categorical columns

In [139]:
num_columns = ['Age at Injury', 
               'IME-4 Count', 
               'Number of Dependents',
               
               'Industry Code_encoded_5. PPD SCH LOSS',
               'Industry Code_encoded_3. MED ONLY',
               'Industry Code_encoded_4. TEMPORARY',
               'Industry Code_encoded_1. CANCELLED', 
               'Industry Code_encoded_8. DEATH',
               'Industry Code_encoded_6. PPD NSL', 
               'Industry Code_encoded_7. PTD',
               
               'WCIO Cause of Injury Code_encoded_5. PPD SCH LOSS',
               'WCIO Cause of Injury Code_encoded_3. MED ONLY',
               'WCIO Cause of Injury Code_encoded_4. TEMPORARY',
               'WCIO Cause of Injury Code_encoded_1. CANCELLED',
               'WCIO Cause of Injury Code_encoded_8. DEATH',
               'WCIO Cause of Injury Code_encoded_6. PPD NSL',
               'WCIO Cause of Injury Code_encoded_7. PTD',
               
               'WCIO Nature of Injury Code_encoded_5. PPD SCH LOSS',
               'WCIO Nature of Injury Code_encoded_3. MED ONLY',
               'WCIO Nature of Injury Code_encoded_4. TEMPORARY',
               'WCIO Nature of Injury Code_encoded_1. CANCELLED',
               'WCIO Nature of Injury Code_encoded_8. DEATH',
               'WCIO Nature of Injury Code_encoded_6. PPD NSL',
               'WCIO Nature of Injury Code_encoded_7. PTD',
               
               'WCIO Part Of Body Code_encoded_5. PPD SCH LOSS',
               'WCIO Part Of Body Code_encoded_3. MED ONLY',
               'WCIO Part Of Body Code_encoded_4. TEMPORARY',
               'WCIO Part Of Body Code_encoded_1. CANCELLED',
               'WCIO Part Of Body Code_encoded_8. DEATH',
               'WCIO Part Of Body Code_encoded_6. PPD NSL',
               'WCIO Part Of Body Code_encoded_7. PTD',
               
               'Industry Code_freq',
               'WCIO Cause of Injury Code_freq', 
               'WCIO Nature of Injury Code_freq',
               'WCIO Part Of Body Code_freq', 
               'Carrier Type_freq',
               'Carrier Name_freq',
               
               'Accident Datemonth',
               'Accident Date_Season_Spring', 
               'Accident Date_Season_Summer',
               'Accident Date_Season_Winter',

               'Days_between_Assembly Date_Accident Date_log',
               'Days_between_C-2 Date_Accident Date_log',
               'Average Weekly Wage Imputed_log']

cat_columns =['Carrier Type_Self-insured Private Entity',
               'Carrier Type_Self-insured Public Entity', 
               'Carrier Type_Special Funds',
               'Carrier Type_State Insurance Fund', 
               'C-3 Date_nabinary', 
               'Average Weekly Wage_nabinary',
               'First Hearing Date_nabinary',
               'Alternative Dispute Resolution_binary',
               'COVID-19 Indicator_binary',
               'Attorney/Representative_binary']


# Create subsets
X_train_num = X_train_encoded[num_columns]
X_train_cat = X_train_encoded[cat_columns]

X_val_num = X_val_encoded[num_columns]
X_val_cat = X_val_encoded[cat_columns]

X_test_num=X_test_encoded[num_columns]
X_test_cat=X_test_encoded[cat_columns]

X_val_final_num = X_val_encoded_final[num_columns]
X_val_final_cat = X_val_encoded_final[cat_columns]

## 3. Scaling

### 3.1 Scale non-binary

> We decided to use normalization because it does not assume any specific distribution of the data. Instead, it scales all features to fit within a specified range—in our case, [0, 1]. This approach can be particularly effective for data that does not follow a Gaussian distribution or has outliers.

In [140]:
scaler = MinMaxScaler().fit(X_train_num)
X_train_num_scaled = scaler.transform(X_train_num)
# print("Parameters fitted:")
# for feature, min_val, max_val in zip(X_train_num.columns, scaler.data_min_, scaler.data_max_):
    # print(f"Variable: {feature} | Min: {min_val} | Max: {max_val}")

# Convert the array to a pandas dataframe
X_train_num_scaled = pd.DataFrame(X_train_num_scaled, columns = X_train_num.columns).set_index(X_train_encoded.index)
# X_train_num_scaled.describe().round(2)

In [141]:
X_val_num_scaled = scaler.transform(X_val_num)
X_val_num_scaled = pd.DataFrame(X_val_num_scaled, columns = X_val_num.columns).set_index(X_val_encoded.index)
# X_val_num_scaled.describe().round(2)

In [142]:
X_test_num_scaled = scaler.transform(X_test_num)
X_test_num_scaled = pd.DataFrame(X_test_num_scaled, columns = X_test_num.columns).set_index(X_test_encoded.index)
# X_test_num_scaled.describe().round(2)

### 3.2 Scale binary

> Peform the same for binary

In [143]:
scaler = MinMaxScaler().fit(X_train_num_bin)
X_train_num_scaled_bin = scaler.transform(X_train_num_bin)
# print("Parameters fitted:")
# for feature, min_val, max_val in zip(X_train_num.columns, scaler.data_min_, scaler.data_max_):
    # print(f"Variable: {feature} | Min: {min_val} | Max: {max_val}")

# Convert the array to a pandas dataframe
X_train_num_scaled_bin = pd.DataFrame(X_train_num_scaled_bin, columns = X_train_num_bin.columns).set_index(X_train_encoded_bin.index)
# X_train_num_scaled.describe().round(2)

In [144]:
X_val_num_scaled_bin = scaler.transform(X_val_num_bin)
X_val_num_scaled_bin = pd.DataFrame(X_val_num_scaled_bin, columns = X_val_num_bin.columns).set_index(X_val_encoded_bin.index)
# X_val_num_scaled.describe().round(2)

In [145]:
X_test_num_scaled_bin = scaler.transform(X_test_num_bin)
X_test_num_scaled_bin = pd.DataFrame(X_test_num_scaled_bin, columns = X_test_num_bin.columns).set_index(X_test_encoded_bin.index)
# X_test_num_scaled.describe().round(2)

## 4. Binary: Feature selection

><strong style="color:#6fa8dc">Add this to a separate python file</strong>


### 4.1 Filter Categorical Features

In [146]:
# squeeze the y_train_bin
y_train_bin = y_train_bin.squeeze()

> Applied without the target being encoded as it avoids introducing false ordinal relationships. Since all 3 (Chi-square, Cramer's V, Chi-Square) can handle categorical data directly, it’s better to not encode it. 

#### 4.1.1 Chi-square

In [147]:
def cramers_v(X, y):
    """
    Calculate Cramér's V for a categorical feature and a target.
    :param X: Categorical feature (Pandas Series or array-like)
    :param y: Target variable (Pandas Series or array-like)
    :return: Cramér's V value
    """
    # Create the contingency table
    df_contingency = pd.crosstab(X, y)

    # Perform chi-square test
    chi2, p, dof, expected = chi2_contingency(df_contingency.values)

    # Calculate Cramér's V
    n = df_contingency.sum().sum()  # Total number of observations
    min_dim = min(df_contingency.shape) - 1  # Min between number of rows and columns - 1
    cramers_v = np.sqrt(chi2 / (n * min_dim))  # Cramér's V formula

    return cramers_v


> These features have **p-value < 0.05** and a **Cramér's V ≥ 0.1**, which indicates:

> - **Strong Statistical Association**: The feature is significantly associated with the target variable.
> - **Practical Importance**: Cramér's V suggests a meaningful strength of association.

In [148]:
# Perform Chi-square test
chi2_values, p_values = chi2(X_train_cat_bin, y_train_bin)

# Create DataFrame for Chi-square results
chi2_results = pd.DataFrame({
    'Column': X_train_cat_bin.columns,
    'Chi2': chi2_values.round(5),
    'p-value': p_values.round(5)
})

# Calculate Cramér's V for binary target
cramers_v_values = []
for var in X_train_cat_bin.columns:
    v = cramers_v(X_train_cat_bin[var], y_train_bin)
    cramers_v_values.append(v)

# Add Cramér's V to DataFrame
chi2_results['Cramér\'s V'] = cramers_v_values

# Filter important features
chi2_important_features = chi2_results[(chi2_results['p-value'] < 0.05) & (chi2_results['Cramér\'s V'] >= 0.1)]

list_features_chi2_cramer = chi2_important_features['Column'].values

print(chi2_important_features)

                              Column         Chi2  p-value  Cramér's V
3  Carrier Type_State Insurance Fund   4015.95429      0.0    0.111341
4                  C-3 Date_nabinary  23479.56453      0.0    0.422936
5       Average Weekly Wage_nabinary  86078.08361      0.0    0.765095
6        First Hearing Date_nabinary  25585.98391      0.0    0.492375
9     Attorney/Representative_binary  77966.98880      0.0    0.532721


#### 4.1.2 Mutual Information

> Given that Mutual Information can measure the dependence between categorical variables and the target. We have decided to keep any features that have mutual informatio > 0.01.

In [149]:
from sklearn.feature_selection import mutual_info_classif

# Calculate mutual information for categorical features
mi_scores = mutual_info_classif(X_train_cat_bin, y_train_bin, discrete_features=True)

# Create DataFrame of results
mi_cat_results = pd.DataFrame({
    'Feature': X_train_cat_bin.columns,
    'Mutual Information': mi_scores
}).sort_values(by='Mutual Information', ascending=False)

mi_cat_results


,Feature,Mutual Information
5,Average Weekly Wage_nabinary,0.365020
9,Attorney/Representative_binary,0.155616
6,First Hearing Date_nabinary,0.135264
4,C-3 Date_nabinary,0.093907
3,Carrier Type_State Insurance Fund,0.006233
7,Alternative Dispute Resolution_binary,0.002891
8,COVID-19 Indicator_binary,0.002841
0,Carrier Type_Self-insured Private Entity,0.000820
1,Carrier Type_Self-insured Public Entity,0.000161
2,Carrier Type_Special Funds,0.000028


In [150]:
mi_important_features = mi_cat_results[mi_cat_results['Mutual Information'] > 0.01]

list_features_mi = mi_cat_results[mi_cat_results['Mutual Information'] > 0.01]['Feature'].values

print(mi_important_features)

                          Feature  Mutual Information
5    Average Weekly Wage_nabinary            0.365020
9  Attorney/Representative_binary            0.155616
6     First Hearing Date_nabinary            0.135264
4               C-3 Date_nabinary            0.093907


#### 4.1.3 Filtered Categorical Features

> Keep if selected by either Chi-square and Cramer's V or Mutual Information

In [151]:
# Select features that appear in at least one of the important feature sets
selected_features = list(set(list_features_chi2_cramer) | set(list_features_mi)) 
selected_features

['First Hearing Date_nabinary',
 'Carrier Type_State Insurance Fund',
 'Average Weekly Wage_nabinary',
 'Attorney/Representative_binary',
 'C-3 Date_nabinary']

In [152]:
selected_features =['Carrier Type_State Insurance Fund',
 'Average Weekly Wage_nabinary',
 'Attorney/Representative_binary',
 'C-3 Date_nabinary',
 'First Hearing Date_nabinary']

In [153]:
# Keep only selected important features in the datasets
X_train_cat_filtered_bin = X_train_cat_bin[selected_features]
X_val_cat_filtered_bin = X_val_cat_bin[selected_features]
X_test_cat_filtered_bin = X_test_cat_bin[selected_features]

### 4.2 Filter Numerical Features

#### 4.2.1 Univariate variance

In [154]:
X_train_num_scaled_bin.var().sort_values(ascending=False)

Accident Date_Season_Summer                     0.190404
Accident Date_Season_Winter                     0.190007
Accident Date_Season_Spring                     0.181340
Carrier Name_freq                               0.142347
WCIO Nature of Injury Code_freq                 0.136746
Industry Code_freq                              0.122438
Number of Dependents                            0.111300
Carrier Type_freq                               0.104407
Accident Datemonth                              0.099277
WCIO Part Of Body Code_freq                     0.098588
Industry Code_encoded_0                         0.095915
Industry Code_encoded_1                         0.095915
WCIO Cause of Injury Code_freq                  0.080295
WCIO Part Of Body Code_encoded_1                0.054324
WCIO Part Of Body Code_encoded_0                0.054324
Average Weekly Wage Imputed_log                 0.049334
WCIO Nature of Injury Code_encoded_1            0.037289
WCIO Nature of Injury Code_enco

> No need to remove any, as there are no features with zero variance.

#### 4.2.2 Spearman Correlation

In [155]:
# Step 1: Calculate the Spearman correlation matrix for numerical features
cor_spearman = X_train_num_scaled_bin.corr(method='spearman')

# Step 2: Flatten the correlation matrix and reset the index
correlation_pairs = cor_spearman.unstack().reset_index()

# Step 3: Rename columns for clarity
correlation_pairs.columns = ['Feature_1', 'Feature_2', 'Correlation']

# Step 4: Filter the table for correlations > 0.8 or < -0.8 and exclude self-correlations (diagonal)
strong_correlations = correlation_pairs[
    ((correlation_pairs['Correlation'] > 0.8) | (correlation_pairs['Correlation'] < -0.8)) & 
    (correlation_pairs['Feature_1'] != correlation_pairs['Feature_2'])
]

# Step 5: Remove duplicate pairs by keeping only one order
strong_correlations = strong_correlations[
    strong_correlations['Feature_1'] < strong_correlations['Feature_2']
]

# Step 6: Sort by correlation value
strong_correlations = strong_correlations.sort_values(by='Correlation', ascending=False)

# Step 7: Display the table
strong_correlations


,Feature_1,Feature_2,Correlation
549,Days_between_Assembly Date_Accident Date_log,Days_between_C-2 Date_Accident Date_log,0.948592
76,Industry Code_encoded_0,Industry Code_encoded_1,-1.000000
126,WCIO Cause of Injury Code_encoded_0,WCIO Cause of Injury Code_encoded_1,-1.000000
176,WCIO Nature of Injury Code_encoded_0,WCIO Nature of Injury Code_encoded_1,-1.000000
226,WCIO Part Of Body Code_encoded_0,WCIO Part Of Body Code_encoded_1,-1.000000


In [156]:
# List of features with strong correlation (correlation > 0.8 or < -0.8)
strong_correlation_features = list(pd.concat([strong_correlations['Feature_2']]).unique())

# Display the list of features with strong correlations
print(strong_correlation_features)


['Days_between_C-2 Date_Accident Date_log', 'Industry Code_encoded_1', 'WCIO Cause of Injury Code_encoded_1', 'WCIO Nature of Injury Code_encoded_1', 'WCIO Part Of Body Code_encoded_1']


> <code>WCIO Part Of Body Code_encoded_0</code> and <code>WCIO Part Of Body Code_encoded_1</code> and the other target encoded variables have -1 correlation, which is to be expected because it was binary encoded. We'll remove one of them. As well as remove <code>Days_between_C-2 Date_Accident Date_log</code> as we believe that <code>	Days_between_Assembly Date_Accident Date_log</code> might be more meaningful since it had no missing values to begin with.

#### 4.2.3 Mutual Information

In [157]:
from sklearn.feature_selection import mutual_info_classif

# Compute mutual information between categorical features and target
mutual_info = mutual_info_classif(X_train_num_scaled_bin, y_train_bin)

# Display features sorted by mutual information
mi_results = pd.DataFrame({
    'Feature': X_train_num_scaled_bin.columns,
    'Mutual Information': mutual_info
}).sort_values(by='Mutual Information', ascending=False)

mi_results

,Feature,Mutual Information
23,Average Weekly Wage Imputed_log,0.378509
1,IME-4 Count,0.156404
13,WCIO Nature of Injury Code_freq,0.054081
8,WCIO Nature of Injury Code_encoded_1,0.053153
7,WCIO Nature of Injury Code_encoded_0,0.052874
16,Carrier Name_freq,0.040810
12,WCIO Cause of Injury Code_freq,0.039775
5,WCIO Cause of Injury Code_encoded_0,0.037862
6,WCIO Cause of Injury Code_encoded_1,0.035482
14,WCIO Part Of Body Code_freq,0.035121


In [158]:
# Assuming `mutual_info` is the result from `mutual_info_classif` function
mi_results = pd.DataFrame({
    'Feature': X_train_num_scaled_bin.columns,
    'Mutual Information': mutual_info
}).sort_values(by='Mutual Information', ascending=False)

# Filter features with Mutual Information > 0.01
mi_important_features = mi_results[mi_results['Mutual Information'] > 0.01]

# Display the important features
print(mi_important_features)


                                 Feature  Mutual Information
23       Average Weekly Wage Imputed_log            0.378509
1                            IME-4 Count            0.156404
13       WCIO Nature of Injury Code_freq            0.054081
8   WCIO Nature of Injury Code_encoded_1            0.053153
7   WCIO Nature of Injury Code_encoded_0            0.052874
16                     Carrier Name_freq            0.040810
12        WCIO Cause of Injury Code_freq            0.039775
5    WCIO Cause of Injury Code_encoded_0            0.037862
6    WCIO Cause of Injury Code_encoded_1            0.035482
14           WCIO Part Of Body Code_freq            0.035121
9       WCIO Part Of Body Code_encoded_0            0.032842
10      WCIO Part Of Body Code_encoded_1            0.031811
15                     Carrier Type_freq            0.025895
11                    Industry Code_freq            0.024014
3                Industry Code_encoded_0            0.021429
4                Industr

> Threshold of 0.01 as well

In [159]:
# List of features with Mutual Information > 0.01
mi_non_important = mi_results[mi_results['Mutual Information'] < 0.01]['Feature'].tolist()
mi_non_important

['Age at Injury',
 'Accident Date_Season_Summer',
 'Accident Date_Season_Winter',
 'Accident Date_Season_Spring',
 'Number of Dependents',
 'Days_between_Assembly Date_Accident Date_log',
 'Accident Datemonth',
 'Days_between_C-2 Date_Accident Date_log']

#### 4.2.3 Filtered Numerical Features

> Drop features that have high correlation and mutual information < 0.01

In [160]:
num_features_to_drop =list(set(num_features_to_drop))

In [161]:
# List of features to drop from X_train_num_scaled
num_features_to_drop = list(set(strong_correlation_features + mi_non_important))

# Drop features
X_train_num_scaled_filtered_bin = X_train_num_scaled_bin.drop(columns=num_features_to_drop)
X_val_num_scaled_filtered_bin = X_val_num_scaled_bin.drop(columns=num_features_to_drop)
X_test_num_scaled_filtered_bin = X_test_num_scaled_bin.drop(columns=num_features_to_drop)

# Verify the remaining columns
print("Remaining features in X_train_num_scaled:", X_train_num_scaled_filtered_bin.columns)


Remaining features in X_train_num_scaled: Index(['IME-4 Count', 'Industry Code_encoded_0',
       'WCIO Cause of Injury Code_encoded_0',
       'WCIO Nature of Injury Code_encoded_0',
       'WCIO Part Of Body Code_encoded_0', 'Industry Code_freq',
       'WCIO Cause of Injury Code_freq', 'WCIO Nature of Injury Code_freq',
       'WCIO Part Of Body Code_freq', 'Carrier Type_freq', 'Carrier Name_freq',
       'Average Weekly Wage Imputed_log'],
      dtype='object')


### 4.3 Combine the Filtered Datasets

> Combine the datasets after dropping

In [162]:
# Combine the filtered datasets
X_train_bin = pd.concat([X_train_cat_filtered_bin, X_train_num_scaled_filtered_bin], axis=1)
X_val_bin =  pd.concat([X_val_cat_filtered_bin, X_val_num_scaled_filtered_bin], axis=1)
X_test_bin =  pd.concat([X_test_cat_filtered_bin, X_test_num_scaled_filtered_bin], axis=1)


# Verify the shape of the combined dataset
print("Shape of combined X_train:", X_train_bin.shape)


Shape of combined X_train: (401797, 17)


> Use this for the binary, as we will be dropping later for multiclass in minority

### 4.4 Feature Selection with All Features

#### 4.4.1 RFE WITH Logistic Regression

> Feature select with RFE and Logistic regression

In [168]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import pandas as pd

# Assuming X_train_bin and y_train_bin are your features and target

sample_size = int(0.5 * len(X_train_bin))  # 10% of the dataset
X_train_sample = X_train_bin.sample(n=sample_size, random_state=42)
y_train_sample = y_train_bin.sample(n=sample_size, random_state=42)

# Step 2: Initialize the logistic regression model
logistic_model = LogisticRegression(max_iter=1000, random_state=42)

# Step 3: Initialize list to store F1 scores for each subset
f1_scores = []

# Step 4: Loop over all subsets of features
for n_features in range(1, X_train_sample.shape[1] + 1):  # For 1 to n features
    # Get the top 'n_features' columns based on importance (or use another criteria)
    selected_features = X_train_sample.columns[:n_features]
    X_train_subset = X_train_sample[selected_features]
    
    # Step 5: Train the model on the subset
    logistic_model.fit(X_train_subset, y_train_sample)
    
    # Step 6: Make predictions and calculate the F1 score
    y_pred = logistic_model.predict(X_val_bin[selected_features])
    f1 = f1_score(y_val_bin, y_pred)
    
    # Store the F1 score
    f1_scores.append((selected_features, f1))

# Step 7: Sort the F1 scores and select the best features
f1_scores.sort(key=lambda x: x[1], reverse=True)  # Sort by F1 score in descending order
selected_features_rfe = f1_scores[0][0]  # Get the subset with the best F1 score

# Step 8: Print the best features based on F1 score
print("Best Features based on F1 score:", selected_features_rfe)


Best Features based on F1 score: Index(['Carrier Type_State Insurance Fund', 'Average Weekly Wage_nabinary',
       'Attorney/Representative_binary', 'C-3 Date_nabinary',
       'First Hearing Date_nabinary', 'IME-4 Count', 'Industry Code_encoded_0',
       'WCIO Cause of Injury Code_encoded_0',
       'WCIO Nature of Injury Code_encoded_0',
       'WCIO Part Of Body Code_encoded_0', 'Industry Code_freq',
       'WCIO Cause of Injury Code_freq', 'WCIO Nature of Injury Code_freq',
       'WCIO Part Of Body Code_freq', 'Carrier Type_freq', 'Carrier Name_freq',
       'Average Weekly Wage Imputed_log'],
      dtype='object')


> keep this for now but can delete after when confirm above

In [ ]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import cross_val_score
# from sklearn.feature_selection import RFE
# import numpy as np

# # Initialize the Logistic Regression model
# log_reg_model = LogisticRegression(max_iter=1000, random_state=42)

# # Sample 1% of the data
# sample_size = int(0.5 * len(X_train_bin))  # 50% of the dataset
# X_train_sample = X_train_bin.sample(n=sample_size, random_state=42)
# y_train_sample = y_train_bin.sample(n=sample_size, random_state=42)

# # Loop over different numbers of features
# feature_counts = range(1, X_train_sample.shape[1] + 1)
# scores = []

# # Loop through each feature count and evaluate the model's performance
# for n in feature_counts:
#     rfe = RFE(log_reg_model, n_features_to_select=n)
#     rfe.fit(X_train_sample, y_train_sample)
    
#     # Get selected features
#     selected_features = X_train_sample.columns[rfe.support_]
    
#     # Evaluate performance using cross-validation with F1 score as the metric
#     score = cross_val_score(log_reg_model, X_train_sample[selected_features], y_train_sample, cv=5, scoring='f1_macro').mean()
#     scores.append(score)

# # Find the number of features that gives the highest F1 score
# best_n_features = feature_counts[np.argmax(scores)]
# print(f"Optimal number of features: {best_n_features}")


In [110]:
# from sklearn.feature_selection import RFE
# from sklearn.linear_model import LogisticRegression
# import pandas as pd

# # Assuming X_train_bin and y_train_bin are your features and target

# # Step 1: Take a smaller sample (e.g., 50% of the data)
# sample_size = int(0.5 * len(X_train_bin))  # 10% of the dataset
# X_train_sample = X_train_bin.sample(n=sample_size, random_state=42)
# y_train_sample = y_train_bin.sample(n=sample_size, random_state=42)

# # Step 2: Initialize Logistic Regression
# logistic_model = LogisticRegression(max_iter=1000, random_state=42)

# # Step 3: Apply RFE to the sample data
# rfe = RFE(logistic_model, n_features_to_select=6)  # Select top 8 features (adjust as needed)
# rfe.fit(X_train_sample, y_train_sample)

# # Step 4: Get the selected features
# selected_features_rfe = X_train_sample.columns[rfe.support_]

# # Print selected features
# print("Selected Features after RFE with Logistic Regression:")
# print(selected_features_rfe)

# # Optionally, you can use the selected features to transform the dataset
# X_train_selected_rfe = X_train_sample[selected_features_rfe]


Selected Features after RFE with Logistic Regression:
Index(['Carrier Type_State Insurance Fund', 'Average Weekly Wage_nabinary',
       'Attorney/Representative_binary', 'First Hearing Date_nabinary',
       'IME-4 Count', 'WCIO Cause of Injury Code_encoded_0',
       'WCIO Nature of Injury Code_encoded_0',
       'WCIO Part Of Body Code_encoded_0', 'Carrier Type_freq',
       'Carrier Name_freq', 'Average Weekly Wage Imputed_log'],
      dtype='object')


#### 4.4.2 Lasso

In [169]:
from sklearn.linear_model import LassoCV
lasso = LassoCV(cv=5)  # Cross-validation to select optimal alpha
lasso.fit(X_train_bin, y_train_bin)
lasso_features = X_train_bin.columns[lasso.coef_ != 0]
print("Selected features by Lasso:", lasso_features)

Selected features by Lasso: Index(['Carrier Type_State Insurance Fund', 'Average Weekly Wage_nabinary',
       'Attorney/Representative_binary', 'C-3 Date_nabinary',
       'First Hearing Date_nabinary', 'IME-4 Count', 'Industry Code_encoded_0',
       'WCIO Cause of Injury Code_encoded_0',
       'WCIO Nature of Injury Code_encoded_0',
       'WCIO Part Of Body Code_encoded_0', 'Industry Code_freq',
       'WCIO Cause of Injury Code_freq', 'WCIO Nature of Injury Code_freq',
       'WCIO Part Of Body Code_freq', 'Carrier Type_freq', 'Carrier Name_freq',
       'Average Weekly Wage Imputed_log'],
      dtype='object')


#### 4.4.3 Ridge

> Find best thrshold for ridge

In [113]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

# Define the model
ridge = Ridge()

# Define the parameter grid
param_grid = {'alpha': [0.001, 0.01, 0.1, 1, 10, 100]}

# Perform grid search with cross-validation
grid_search = GridSearchCV(ridge, param_grid, cv=5)
grid_search.fit(X_train_bin, y_train_bin)

# Get the best alpha
best_alpha = grid_search.best_params_['alpha']
print(f"Best alpha value: {best_alpha}")


Best alpha value: 0.1


In [ ]:
from sklearn.linear_model import RidgeCV
import numpy as np

# Initialize Ridge regression model with cross-validation
ridge_model = RidgeCV(cv=5)

# Fit the model on the training data
ridge_model.fit(X_train_bin, y_train_bin)

# Get the feature coefficients
coefficients = ridge_model.coef_

# Set a threshold to select features (e.g., absolute coefficient > 0.1)
threshold = 0.1
ridge_features = X_train_bin.columns[np.abs(coefficients) > threshold]

print(f"Selected features using Ridge regression: {ridge_features}")


Selected features using Ridge regression: Index(['Carrier Type_State Insurance Fund', 'Average Weekly Wage_nabinary',
       'Attorney/Representative_binary', 'IME-4 Count',
       'WCIO Nature of Injury Code_encoded_0', 'Carrier Name_freq',
       'Average Weekly Wage Imputed_log'],
      dtype='object')


#### 4.4.4 Random Forest Important Features

> Find best alpha

In [115]:
from sklearn.ensemble import RandomForestClassifier

# Initialize the Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=47)

# Fit the model on X_train and y_train_encoded
rf_model.fit(X_train_bin, y_train_bin)

# Get feature importances
importances = rf_model.feature_importances_

# Sort feature importances in descending order
important_indices = importances.argsort()[::-1]

# Print the feature importances
print("Feature importances:", importances)
print("Sorted feature indices:", important_indices)


Feature importances: [0.00211268 0.2846026  0.08045238 0.02492534 0.03600001 0.05694644
 0.02339744 0.03569198 0.02276218 0.03082299 0.02198453 0.03080075
 0.01506359 0.02924714 0.00888832 0.07782569 0.21847593]
Sorted feature indices: [ 1 16  2 15  5  4  7  9 11 13  3  6  8 10 12 14  0]


In [116]:
import numpy as np

# Calculate cumulative sum of feature importances
cumulative_importance = np.cumsum(importances[important_indices])

# Find the index where cumulative importance exceeds 90%
threshold = 0.90
index_90 = np.argmax(cumulative_importance >= threshold)

# The number of features that explain 90% of the importance
num_features_90 = index_90 + 1  # Adding 1 since index starts at 0

print(f"Number of features explaining 90% of cumulative importance: {num_features_90}")

# Select the top features based on this number
random_forest_features = X_train_bin.columns[important_indices[:num_features_90]]
print(f"Selected features that explain 90% of cumulative importance: {random_forest_features}")


Number of features explaining 90% of cumulative importance: 11
Selected features that explain 90% of cumulative importance: Index(['Average Weekly Wage_nabinary', 'Average Weekly Wage Imputed_log',
       'Attorney/Representative_binary', 'Carrier Name_freq', 'IME-4 Count',
       'First Hearing Date_nabinary', 'WCIO Cause of Injury Code_encoded_0',
       'WCIO Part Of Body Code_encoded_0', 'WCIO Cause of Injury Code_freq',
       'WCIO Part Of Body Code_freq', 'C-3 Date_nabinary'],
      dtype='object')


#### 4.4.5 Voting of Best Features

In [170]:
# Convert all feature sets to sets
rfe_set = set(selected_features_rfe)
lasso_set = set(lasso_features)
ridge_set = set(ridge_features)
rf_set = set(random_forest_features)

# Find features selected by at least three methods
final_features_binary_set = (
    (lasso_set & rfe_set & ridge_set) |
    (lasso_set & ridge_set & rf_set) |
    (ridge_set & rfe_set & rf_set)
)

# Convert the final features set to a list (optional, for easier use later)
final_features_binary = list(final_features_binary_set)

# Print the selected features
print(len(final_features_binary))
print("Features selected by at least three methods:")
final_features_binary


7
Features selected by at least three methods:


['Carrier Type_State Insurance Fund',
 'Carrier Name_freq',
 'Average Weekly Wage_nabinary',
 'Attorney/Representative_binary',
 'IME-4 Count',
 'Average Weekly Wage Imputed_log',
 'WCIO Nature of Injury Code_encoded_0']

In [171]:
X_train_bin_filter = X_train_bin[final_features_binary]
X_val_bin_filter = X_val_bin[final_features_binary]
X_test_bin_filter = X_test_bin[final_features_binary]

## 5. Binary: Model

In [172]:
def custom_loss(y_pred, dtrain):
    """
    Custom loss function penalizing false negatives for class 0.
    Parameters:
    - y_pred: Predicted values (log-odds).
    - dtrain: DMatrix containing true labels.

    Returns:
    - grad: Gradient of the loss.
    - hess: Hessian of the loss.
    """
    y_true = dtrain.get_label()  # Extract true labels from DMatrix
    y_pred = 1 / (1 + np.exp(-y_pred))  # Convert log-odds to probabilities

    # Adjust weights for class 0
    weight = np.where(y_true == 0, 2.0, 1.0)  # Double penalty for class 0 FN

    grad = (y_pred - y_true) * weight  # Gradient
    hess = y_pred * (1 - y_pred) * weight  # Hessian
    return grad, hess


In [ ]:
import xgboost as xgb

# Convert data to DMatrix format
dtrain = xgb.DMatrix(X_train_bin_filter, label=y_train_bin)
dval = xgb.DMatrix(X_val_bin_filter, label=y_val_bin)

# Parameters for XGBoost
params = {
    "objective": "binary:logistic",  # Use binary classification
    "eval_metric": "logloss",  # Standard evaluation metric
    "eta": 0.1,  # Learning rate
    "max_depth": 6,  # Maximum depth of trees
}

# Train the model with custom loss function
binary_model = xgb.train(
    params,
    dtrain,
    num_boost_round=100,
    obj=custom_loss,  # Custom loss function
    evals=[(dval, "validation")],
    early_stopping_rounds=10,  # Stop if no improvement
)


In [ ]:
# Predict probabilities
y_proba = binary_model.predict(xgb.DMatrix(X_val_bin_filter))

# Convert probabilities to binary predictions using a threshold (default is 0.5)
threshold = 0.5
y_pred_binary = (y_proba > threshold).astype(int)

# Evaluate
from sklearn.metrics import classification_report, accuracy_score, f1_score

print("Accuracy:", accuracy_score(y_val_bin, y_pred_binary))
print("F1 Score:", f1_score(y_val_bin, y_pred_binary, average="macro"))
print(classification_report(y_val_bin, y_pred_binary))


In [173]:
import xgboost as xgb

binary_model = xgb.XGBClassifier(eval_metric="mlogloss")
binary_model.fit(X_train_bin_filter, y_train_bin)

# 3. Make predictions
y_pred_binary = binary_model.predict(X_val_bin_filter)  # Assuming X_test is available


# Evaluate the model
accuracy = accuracy_score(y_val_bin, y_pred_binary)
f1_macro = f1_score(y_val_bin, y_pred_binary, average='macro')  # Macro-averaged F1 score

# Print individual model results
print(f"XGBClassifier Accuracy: {accuracy:.4f}")
print(f"XGBClassifier Macro F1 Score: {f1_macro:.4f}")
print(classification_report(y_val_bin, y_pred_binary))  # Detailed report including precision, recall, and F1 score per class
print("-" * 50)

XGBClassifier Accuracy: 0.8974
XGBClassifier Macro F1 Score: 0.8968
              precision    recall  f1-score   support

           0       0.96      0.83      0.89     84881
           1       0.85      0.96      0.90     87319

    accuracy                           0.90    172200
   macro avg       0.90      0.90      0.90    172200
weighted avg       0.90      0.90      0.90    172200

--------------------------------------------------


In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report

# Ensure target variable is 1D
y_train_bin = y_train_bin.values.ravel() if hasattr(y_train_bin, "values") else y_train_bin
y_val_bin = y_val_bin.values.ravel() if hasattr(y_val_bin, "values") else y_val_bin

# Define base models (XGBoost and CatBoost in this case)
estimators = [
    ('xgb', XGBClassifier(random_state=42, eval_metric="mlogloss")),   
    ('catboost', CatBoostClassifier(iterations=1000, random_state=42, verbose=100, early_stopping_rounds=50))  
]

# Define final estimator (Logistic Regression in this case)
final_estimator = LogisticRegression(max_iter=500, random_state=42)

# Create the Stacking Classifier model
stacking_model = StackingClassifier(estimators=estimators, final_estimator=final_estimator, n_jobs=-1)

# Train the stacking model
stacking_model.fit(X_train_bin_filter, y_train_bin)

# Predict on validation set and evaluate performance
y_pred = stacking_model.predict(X_val_bin_filter)
print(classification_report(y_val_bin, y_pred))


## 6. Multiclass: Feature Selection

### 6.1 Filter Categorical Features

#### 6.1.1 Chi-square & Cramer's V

In [ ]:
# Perform Chi-square test
chi2_values, p_values = chi2(X_train_cat, y_train_not_encoded)

# Create DataFrame for Chi-square results
chi2_results = pd.DataFrame({
    'Column': X_train_cat.columns,
    'Chi2': chi2_values.round(5),
    'p-value': p_values.round(5)
})

# Calculate Cramér's V for binary target
cramers_v_values = []
for var in X_train_cat.columns:
    v = cramers_v(X_train_cat[var], y_train_not_encoded)
    cramers_v_values.append(v)

# Add Cramér's V to DataFrame
chi2_results['Cramér\'s V'] = cramers_v_values

# Filter important features
chi2_important_features = chi2_results[(chi2_results['p-value'] < 0.05) & (chi2_results['Cramér\'s V'] >= 0.1)]

list_features_chi2_cramer = chi2_important_features['Column'].values

print(chi2_important_features)

#### 6.1.2 Mutual Information

In [ ]:
# from sklearn.feature_selection import mutual_info_classif

# # Calculate mutual information for categorical features
# mi_scores = mutual_info_classif(X_train_cat, y_train_not_encoded, discrete_features=True)

# # Create DataFrame of results
# mi_results = pd.DataFrame({
#     'Feature': X_train_cat.columns,
#     'Mutual Information': mi_scores
# }).sort_values(by='Mutual Information', ascending=False)

# mi_results

# mi_important_features = mi_results[mi_results['Mutual Information'] > 0.05]

# list_features_mi = mi_results[mi_results['Mutual Information'] > 0.05]['Feature'].values

# print(mi_important_features)

#### 6.1.3 Filtered Categorical Features

In [ ]:
# # Select features that appear in at least one of the important feature sets
# selected_features = list(set(list_features_chi2_cramer) | set(list_features_mi)) 
# print(selected_features)

# # Keep only selected important features in the datasets
# X_train_cat_filtered = X_train_cat[selected_features]
# X_val_cat_filtered = X_val_cat[selected_features]
# X_test_cat_filtered = X_test_cat[selected_features]

In [110]:
selected_features = ['COVID-19 Indicator_binary', 'Carrier Type_Self-insured Public Entity', 'Average Weekly Wage_nabinary', 'C-3 Date_nabinary', 'First Hearing Date_nabinary', 'Attorney/Representative_binary']
X_train_cat_filtered = X_train_cat[selected_features]
X_val_cat_filtered = X_val_cat[selected_features]
X_test_cat_filtered = X_test_cat[selected_features]

### 6.2 Filter Numerical

#### 6.2.1 Univariate variance

In [ ]:
# X_train_num_scaled.var().sort_values(ascending=False)

> No features with variance zero. Don't drop any here.

#### 6.2.2 Spearman Correlation

In [ ]:
# # Step 1: Calculate the Spearman correlation matrix for numerical features
# cor_spearman = X_train_num_scaled.corr(method='spearman')

# # Step 2: Flatten the correlation matrix and reset the index
# correlation_pairs = cor_spearman.unstack().reset_index()

# # Step 3: Rename columns for clarity
# correlation_pairs.columns = ['Feature_1', 'Feature_2', 'Correlation']

# # Step 4: Filter the table for correlations > 0.8 or < -0.8 and exclude self-correlations (diagonal)
# strong_correlations = correlation_pairs[
#     ((correlation_pairs['Correlation'] > 0.9) | (correlation_pairs['Correlation'] < -0.9)) & 
#     (correlation_pairs['Feature_1'] != correlation_pairs['Feature_2'])
# ]

# # Step 5: Remove duplicate pairs by keeping only one order
# strong_correlations = strong_correlations[
#     strong_correlations['Feature_1'] < strong_correlations['Feature_2']
# ]

# # Step 6: Sort by correlation value
# strong_correlations = strong_correlations.sort_values(by='Correlation', ascending=False)

# # Step 7: Display the table
# strong_correlations


#### 6.2.3 Mutual Information

In [ ]:
# from sklearn.feature_selection import mutual_info_classif

# # Compute mutual information between categorical features and target
# mutual_info = mutual_info_classif(X_train_num_scaled, y_train_encoded)

# # Display features sorted by mutual information
# mi_results = pd.DataFrame({
#     'Feature': X_train_num_scaled.columns,
#     'Mutual Information': mutual_info
# }).sort_values(by='Mutual Information', ascending=False)

# mi_results

In [ ]:
# # Assuming `mutual_info` is the result from `mutual_info_classif` function
# mi_results = pd.DataFrame({
#     'Feature': X_train_num_scaled.columns,
#     'Mutual Information': mutual_info
# }).sort_values(by='Mutual Information', ascending=False)

# # Filter features with Mutual Information > 0.05
# mi_important_features = mi_results[mi_results['Mutual Information'] > 0.01]

# # Display the important features
# print(mi_important_features)


#### 6.2.3 Filtered Numerical Features

In [ ]:
# List of features to drop from X_train_num_scaled
num_features_to_drop = [
    'Days_between_C-2 Date Imputed_Accident Date_log'
    ]

# Drop features
X_train_num_scaled_filtered = X_train_num_scaled.drop(columns=num_features_to_drop)
X_val_num_scaled_filtered = X_val_num_scaled.drop(columns=num_features_to_drop)
X_test_num_scaled_filtered = X_test_num_scaled.drop(columns=num_features_to_drop)

# Verify the remaining columns
print("Remaining features in X_train_num_scaled:", X_train_num_scaled_filtered.columns)
print(len(X_train_num_scaled_filtered.columns))


### 6.3 Combine the Filtered Datasets

In [ ]:
# Combine the filtered datasets
X_train = pd.concat([X_train_cat_filtered, X_train_num_scaled_filtered], axis=1)
X_val =  pd.concat([X_val_cat_filtered, X_val_num_scaled_filtered], axis=1)
X_test =  pd.concat([X_test_cat_filtered, X_test_num_scaled_filtered], axis=1)


# Verify the shape of the combined dataset
print("Shape of combined X_train:", X_train.shape)
print("Shape of combined X_train:", X_val.shape)


### 6.4 Feature Selection All Features

#### 6.4.1 Lasso

In [ ]:
from sklearn.linear_model import LassoCV
lasso = LassoCV(cv=5)  # Cross-validation to select optimal alpha
lasso.fit(X_train, y_train_encoded)
lasso_features = X_train.columns[lasso.coef_ != 0]
print("Selected features by Lasso:", lasso_features)

#### 6.4.2 Ridge

In [ ]:
from sklearn.linear_model import RidgeCV
import numpy as np

# Initialize Ridge regression model with cross-validation
ridge_model = RidgeCV(cv=5)

# Fit the model on the training data
ridge_model.fit(X_train, y_train_encoded)

# Get the feature coefficients
coefficients = ridge_model.coef_

# Set a threshold to select features (e.g., absolute coefficient > 0.01)
threshold = 0.01
ridge_features = X_train.columns[np.abs(coefficients) > threshold]

print(f"Selected features using Ridge regression: {ridge_features}")


#### 6.4.3 Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Initialize the Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=47)

# Fit the model on X_train and y_train_encoded
rf_model.fit(X_train, y_train_encoded)

# Get feature importances
importances = rf_model.feature_importances_

# Sort feature importances in descending order
important_indices = importances.argsort()[::-1]

# Print the feature importances
print("Feature importances:", importances)
print("Sorted feature indices:", important_indices)


In [ ]:
import numpy as np

# Calculate cumulative sum of feature importances
cumulative_importance = np.cumsum(importances[important_indices])

# Find the index where cumulative importance exceeds 90%
threshold = 0.90
index_90 = np.argmax(cumulative_importance >= threshold)

# The number of features that explain 90% of the importance
num_features_90 = index_90 + 1  # Adding 1 since index starts at 0

print(f"Number of features explaining 90% of cumulative importance: {num_features_90}")

# Select the top features based on this number
random_forest_features = X_train.columns[important_indices[:num_features_90]]
print(f"Selected features that explain 90% of cumulative importance: {random_forest_features}")


In [ ]:
import pandas as pd
from xgboost import XGBClassifier

# Initialize the XGBoost Classifier without 'use_label_encoder'
xgb_model = XGBClassifier(n_estimators=100, random_state=47, eval_metric="mlogloss")

# Fit the model on X_train and y_train_encoded
xgb_model.fit(X_train, y_train_encoded)

# Get feature importances
importances = xgb_model.feature_importances_

# Create a DataFrame with feature names and their importances
feature_importance_df = pd.DataFrame({
    'Feature': X_train.columns,  # Assumes X_train is a pandas DataFrame
    'Importance': importances
})

# Sort the DataFrame by importance in descending order
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Print the sorted feature names and their importance
print(feature_importance_df)


In [ ]:
import numpy as np
import pandas as pd

# Calculate cumulative sum of feature importances
cumulative_importance = np.cumsum(feature_importance_df['Importance'])

# Find the index where cumulative importance exceeds 90%
threshold = 0.90
index_90 = np.argmax(cumulative_importance >= threshold)

# The number of features that explain 90% of the importance
num_features_90 = index_90 + 1  # Adding 1 since index starts at 0

print(f"Number of features explaining 90% of cumulative importance: {num_features_90}")

# Select the top features based on this number
xgboost_features = feature_importance_df['Feature'].iloc[:num_features_90]
print(f"Selected features that explain 90% of cumulative importance:\n{xgboost_features}")


#### 6.4.5 Voting for the Best Feature

In [ ]:
# Convert all feature sets to sets
lasso_set = set(lasso_features)
ridge_set = set(ridge_features)
rf_set = set(random_forest_features)
xgb_set = set(xgboost_features)

# Find features selected by at least three methods
final_features_set = (
    (lasso_set & ridge_set & rf_set) |  # Features in all three: Lasso, Ridge, RF
    (lasso_set & ridge_set & xgb_set) |  # Features in Lasso, Ridge, XGBoost
    (lasso_set & rf_set & xgb_set) |     # Features in Lasso, RF, XGBoost
    (ridge_set & rf_set & xgb_set)      # Features in Ridge, RF, XGBoost
)

# Convert the final features set to a list (optional, for easier use later)
final_features = list(final_features_set)

# Print the selected features
print(len(final_features))
print("Features selected by at least three methods:")
print(final_features)


> Apply to datasets

In [ ]:
import pandas as pd

# Assume the following sets represent the selected features from each method
lasso_set = set(lasso_features)
ridge_set = set(ridge_features)
rf_set = set(random_forest_features)
xgb_set = set(xgboost_features)

# Combine all unique features from all sets
all_features = list(set(lasso_set) | set(ridge_set) | set(rf_set) | set(xgb_set))

# Create a DataFrame to show the feature selection results
feature_selection_table = pd.DataFrame({
    'Feature': all_features,
    'Lasso': [feature in lasso_set for feature in all_features],
    'Ridge': [feature in ridge_set for feature in all_features],
    'Random Forest': [feature in rf_set for feature in all_features],
    'XGBoost': [feature in xgb_set for feature in all_features]
})

# Add a column to count how many methods selected each feature
feature_selection_table['Selected by 2 or more'] = (
    feature_selection_table[['Lasso', 'Ridge', 'Random Forest', 'XGBoost']].sum(axis=1) >= 3
)

# Extract lists of features selected by each method
lasso_selected_features = feature_selection_table[feature_selection_table['Lasso']]['Feature'].tolist()
ridge_selected_features = feature_selection_table[feature_selection_table['Ridge']]['Feature'].tolist()
rf_selected_features = feature_selection_table[feature_selection_table['Random Forest']]['Feature'].tolist()
xgb_selected_features = feature_selection_table[feature_selection_table['XGBoost']]['Feature'].tolist()

# Extract features selected by 3 or more methods
selected_by_3_or_more = feature_selection_table[feature_selection_table['Selected by 2 or more']]['Feature'].tolist()

# Function to highlight selected features for Lasso, Ridge, RF, and XGBoost
def highlight_selected(val):
    return 'background-color: #aed6f1' if val else ''

def highlight_selected_by_3_or_more(val):
    return 'background-color: #abebc6' if val else ''

# Apply the styling to each column using apply and map
styled_table = feature_selection_table.style.apply(
    lambda x: ['background-color: #aed6f1' if val else '' for val in x], 
    subset=['Lasso', 'Ridge', 'Random Forest', 'XGBoost']
).apply(
    lambda x: ['background-color: #abebc6' if val else '' for val in x], 
    subset=['Selected by 2 or more']
)

# Display the styled table
styled_table


In [ ]:
selected_by_3_or_more = ['Accident Datemonth',
 'Industry Code_encoded_4. TEMPORARY',
 'WCIO Part Of Body Code_encoded_5. PPD SCH LOSS',
 'Average Weekly Wage Imputed_log',
 'C-3 Date_nabinary',
 'WCIO Cause of Injury Code_encoded_3. MED ONLY',
 'COVID-19 Indicator_binary',
 'Industry Code_encoded_3. MED ONLY',
 'Age at Injury',
 'WCIO Part Of Body Code_freq',
 'WCIO Cause of Injury Code_freq',
 'WCIO Nature of Injury Code_encoded_5. PPD SCH LOSS',
 'IME-4 Count',
 'WCIO Cause of Injury Code_encoded_1. CANCELLED',
 'WCIO Cause of Injury Code_encoded_6. PPD NSL',
 'WCIO Part Of Body Code_encoded_4. TEMPORARY',
 'WCIO Cause of Injury Code_encoded_4. TEMPORARY',
 'WCIO Nature of Injury Code_encoded_1. CANCELLED',
 'Industry Code_freq',
 'Days_between_Assembly Date_Accident Date_log',
 'WCIO Part Of Body Code_encoded_8. DEATH',
 'WCIO Cause of Injury Code_encoded_5. PPD SCH LOSS',
 'Carrier Type_Self-insured Public Entity',
 'Carrier Name_freq',
 'WCIO Nature of Injury Code_encoded_3. MED ONLY',
 'Average Weekly Wage_nabinary',
 'Carrier Type Imputed_freq',
 'Attorney/Representative_binary',
 'WCIO Part Of Body Code_encoded_6. PPD NSL',
 'First Hearing Date_nabinary',
 'Industry Code_encoded_1. CANCELLED',
 'WCIO Part Of Body Code_encoded_3. MED ONLY',
 'WCIO Nature of Injury Code_encoded_6. PPD NSL',
 'WCIO Nature of Injury Code_encoded_4. TEMPORARY',
 'Industry Code_encoded_5. PPD SCH LOSS']

In [131]:
X_train_filter = X_train[selected_by_3_or_more]
X_val_filter = X_val[selected_by_3_or_more]
X_test_filter = X_test[selected_by_3_or_more]

## 7. Multiclass Model

In [505]:
from imblearn.combine import SMOTETomek 


In [ ]:
# Assuming `y` is your target variable
class_distribution = y_train_encoded.value_counts()
class_percentages = y_train_encoded.value_counts(normalize=True) * 100

# Combine into a single DataFrame for better visualization
distribution_df = pd.DataFrame({
    'Class': class_distribution.index,
    'Count': class_distribution.values,
    'Percentage (%)': class_percentages.values
})
print(distribution_df)


In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler



# Define undersampling strategy based on the actual class distribution
undersample_strategy = {
    0: 50621,
    2: 47591,
    1: 33513,
    4: 8191,
    4: 2924,
    5: 324,
    6:67
}

# Apply undersampling to majority classes
undersampler = RandomUnderSampler(sampling_strategy=undersample_strategy, random_state=42)
X_resampled, y_resampled = undersampler.fit_resample(X_train_filter, y_train_encoded)

# # Define SMOTE strategy for oversampling the minority classes
# smote = SMOTE(sampling_strategy={
#     7:807,

#     6: 670,   # Oversample class '5' to 3,000
# }, random_state=42)

# # Apply SMOTE to oversample minority classes
# X_resampled, y_resampled = smote.fit_resample(X_resampled, y_resampled)

# Check the new class distribution after resampling
unique_classes_final, class_counts_final = np.unique(y_resampled, return_counts=True)
class_distribution_final = dict(zip(unique_classes_final, class_counts_final))
print("New Class Distribution after Resampling:", class_distribution_final)

In [255]:
# weights = np.ones(len(y_train_encoded))
# weights[y_train_encoded == 0] = 10


weights = np.ones(len(y_resampled))
weights[y_resampled == 0] = 2

In [ ]:
import xgboost as xgb

multiclass_model = xgb.XGBClassifier(eval_metric="mlogloss")
# multiclass_model.fit(X_train_filter, y_train_encoded, sample_weight=weights)
multiclass_model.fit(X_resampled, y_resampled, sample_weight=weights)

# 3. Make predictions
y_pred = multiclass_model.predict(X_val_filter)  # Assuming X_test is available


# Evaluate the model
accuracy = accuracy_score(y_val_encoded, y_pred)
f1_macro = f1_score(y_val_encoded, y_pred, average='macro')  # Macro-averaged F1 score

# Print individual model results
print(f"XGBClassifier Accuracy: {accuracy:.4f}")
print(f"XGBClassifier Macro F1 Score: {f1_macro:.4f}")
print(classification_report(y_val_encoded, y_pred))  # Detailed report including precision, recall, and F1 score per class
print("-" * 50)

## 7. Two-stage Model prediction

In [257]:
# Predict probabilities for binary classification (whether it's non-comp or not)
y_proba_binary = stacking_model.predict(X_test_bin_filter)

# y_proba_binary = binary_model.predict(xgb.DMatrix(X_test_bin_filter))

# Convert probabilities to binary predictions (0 for not non-comp, 1 for non-comp)
threshold = 0.5  # Default threshold, you can adjust based on recall
y_pred_binary = (y_proba_binary > threshold).astype(int)


In [258]:
# Get the indices of instances predicted as not non-comp (i.e., predicted as 0)
non_com =  (y_pred_binary == 1)
not_non_comp_indices = (y_pred_binary == 0)

# Use these indices to filter `X_val_filter` to get the corresponding instances
X_test_filtered_for_multiclass = X_test_filter[not_non_comp_indices]


In [259]:
# Predict multiclass labels for the filtered data (instances not predicted as non-comp)
y_pred_multiclass = multiclass_model.predict(X_test_filtered_for_multiclass)


In [ ]:
# Assuming X_test_filtered_for_multiclass is a DataFrame, and y_pred_multiclass are the predictions
X_test_filtered_for_multiclass.loc[:, 'predictions'] = y_pred_multiclass

# Now you have a DataFrame with the original features and the predictions as a new column
X_test_filtered_for_multiclass.head()


In [ ]:
# Get the indices of the filtered dataset (i.e., the rows of X_test that were used for multiclass prediction)
filtered_indices = X_test_filtered_for_multiclass.index

# Now, you can map the predictions back to the original dataset by creating a new column in the original DataFrame
X_test.loc[filtered_indices, 'predictions'] = y_pred_multiclass

# Now you have the predictions in the original dataset at the corresponding rows
X_test

In [ ]:
# Analyze the distribution of predicted classes
predictions_df = pd.DataFrame(y_pred_multiclass, columns=["Predicted Label"])
print("Predicted Class Distribution:")
print(predictions_df["Predicted Label"].value_counts(normalize=True))


In [263]:
claim_injury_type_mapping = {
    '4. TEMPORARY': 0,
    '5. PPD SCH LOSS': 1,
    '3. MED ONLY': 2,
    '6. PPD NSL': 3,
    '1. CANCELLED': 4,
    '8. DEATH': 5,
    '7. PTD': 6
}


# Reverse the mapping
reverse_claim_injury_type_mapping = {v: k for k, v in claim_injury_type_mapping.items()}

# Use the reversed mapping to get the original labels
X_test['predictions'] = X_test['predictions'].map(reverse_claim_injury_type_mapping)


In [264]:
# Fill NaN values in the 'predictions' column with the value '2. NON-COMP'
X_test['predictions'] = X_test['predictions'].fillna('2. NON-COMP')

In [265]:
X_test['predictions'].to_csv('test_predictions.csv')